In [1]:
from alphatoe import game, data, train, evals
import torch
from torch.nn.functional import cross_entropy
from tqdm import tqdm
import einops

In [2]:
# Combine all data to get loss across all games
train_data, train_labels, test_data, test_labels = data.gen_data("all")
all_data = torch.cat([train_data, test_data])
all_labels = torch.cat([train_labels, test_labels])

Generating all possible games...
Generated 255168 games
Generated array of moves
torch.Size([255168, 10])
Generated data and labels
One hot encoded labels


In [3]:
# Drop the dummy dimension here
logits = einops.rearrange(torch.stack([game.autoregressive_guess(seq)[0] for seq in tqdm(all_data) ]), 'batch seq tokens -> (batch seq) tokens')
arranged_labels = train.rearrange(all_labels)

100%|██████████| 255168/255168 [01:53<00:00, 2246.89it/s]


In [4]:

loss = cross_entropy(torch.softmax(logits, dim=1), arranged_labels[0])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [5]:
logits[0].shape

torch.Size([10])

In [ ]:
print(logits[0])
print(arranged_labels[0])

tensor([0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111,
          -inf])
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])


In [ ]:
print(loss)

tensor(nan)


In [ ]:
sampled_games = evals.sample_games(game.autoregressive_guess, temp=1, num_games = 10000)

100%|██████████| 10000/10000 [00:20<00:00, 496.45it/s]


In [ ]:
error_rate = evals.error_rate(sampled_games)
print(error_rate)


0.4404
